In [1]:
# Step1: Generate each_contest_before_csv
import xml.etree.ElementTree as ET
import pandas as pd  

# Input will be the card_ID 
cols = ["姓名", "車號", "車隊", "車型", "日期"]
rows = []

#Parse XML file
tree = ET.parse('race_cars_info.xml')
root = tree.getroot()
for elem in root:
        name = elem.find("姓名").text
        car_label = elem.find("車號").text
        car_team = elem.find("車隊").text
        car_model = elem.find("車型").text
        date = elem.find("日期").text 
            
        rows.append({"姓名": name, 
                  "車號": car_label,
                  "車隊": car_team,
                  "車型": car_model,
                  "日期": date})

df = pd.DataFrame(rows, columns = cols) 
  
# # write dataframe to csv
df.to_csv('each_contest_before.csv',index=False)

In [2]:
# Step2: Generate each_contest_after_csv

df = pd.read_csv ('each_contest_before.csv')
df['車競賽'] = 1

# After all five scores are inserted. 
scores_1 = [45, 44, 49]
df['第一圈秒數'] = scores_1

scores_2 = [42, 44, 41]
df['第二圈秒數'] = scores_2

scores_3 = [39, 50, 35]
df['第三圈秒數'] = scores_3

scores_4 = [44, 49, 44]
df['第四圈秒數'] = scores_4

scores_5 = [49, 42, 31]
df['第五圈秒數'] = scores_5

df2 = df
df2 = df2.drop(columns=['車競賽', '姓名','車號', '車隊', '日期', '車型'])
best_score_label = df2.idxmin(axis = 1)

best_scores = []
i = 0
for ind in best_score_label:
    best_scores.append(df2.iloc[i][ind])
    i = i + 1
    
df['最佳圈秒數'] = best_scores
df.to_csv('each_contest_after.csv', index=False)

In [3]:
# Step3: Convert csv to xml

import csv
csvFile = 'each_contest_after.csv'
xmlFile = 'result.xml'

csvData = csv.reader(open(csvFile))
xmlData = open(xmlFile, 'w')
xmlData.write('<?xml version="1.0"?>' + "\n")
# there must be only one top-level tag
xmlData.write('<csv_data>' + "\n")

rowNum = 0
for row in csvData:
    if rowNum == 0:
        tags = row
        # replace spaces w/ underscores in tag names
        for i in range(len(tags)):
            tags[i] = tags[i].replace(' ', '_')
    else: 
        xmlData.write('<row>' + "\n")
        for i in range(len(tags)):
            xmlData.write('    ' + '<' + tags[i] + '>' \
                          + row[i] + '</' + tags[i] + '>' + "\n")
        xmlData.write('</row>' + "\n")
            
    rowNum +=1

xmlData.write('</csv_data>' + "\n")
xmlData.close()